In [ ]:
class KmeansInterp(KMeans):

  def __init__(self, ordered_feature_names, feature_importance_method='wcss_min', **kwargs) :
    super(KMeansInterp, self).__init__(**kwargs)         ## call kmeanns on given dataframe
    self.feature_importance_method = feature_importance_method
    self.ordered_feature_names = ordered_feature_names


  def fit(self, X, y=None, sample_weight=None):

    super().fit(X=x, y=y, sample_weight=sample_weight)

    if self.feature_importance_method = "wcss_min":
      self.feature_importances = self.get_feature_wcss_min()

    if self.feature_importance_method = "unsup2sup":
      self.feature_importances = self.get_feature_imp_unsup2sup(X)

    else:
      raise Exception(f"{self.feature_importances_method} is not available")


  def fit_predict(self, X, y=None, sample_weight=None):
    return super().fit_predict(X=X, y=y, sample_weight=sample_weight)

  def predict(self, X, sample_weight=None)

    self.labels_= super().predict(X=X, sample_weight = sample_weight)

    if self.feature_importance_method = "wcss_min":
      self.feature_importances = self.get_feature_wcss_min()

    if self.feature_importance_method = "unsup2sup":
      self.feature_importances = self.get_feature_imp_unsup2sup(X)

    else:
      raise Exception(f"{self.feature_importances_method} is not available")


  def get_feature_wcss_min(self):

    labels = self.n_clusters
    centroids = self.cluster_centers_
    centroids = np.vectorize(lambda x: np.abs(x))(centroids)
    sorted_centroid_features_idx = centroids.argsort(axis=1)[:,::-1]        ##sorted feature index for each centroid

    cluster_feature_weights = {}

    for label, centriod in zip(range(labels), sorted_centroid_features_idx):   ### get weight of  each feature which is distance moved by centroid by a feature of wcss MINIMIZATION, highter the disance moved, higher the impact of feature
        ordered_cluster_feature_weights = centroids[label][sorted_centroid_features_idx[label]]
        ordered_cluster_features = [self.ordered_feature_names[feature] for feature in centroid]
        cluster_feature_weights[label] = list(zip(ordered_cluster_features, list(np.around(np.array(ordered_cluster_feature_weights),2))))


    return cluster_feature_weights



    def get_feature_imp_unsup2sup(self,X):

      try:
        from sklearn.ensemble import RandoForestClassifier

      except ImportError as IE:

        raise Exception("Please install")

      cluster_feature_weights = {}

      for label in range(self.n_clusters):
        binary_enc = np.vectorize(lambda x:1 if x == label else 0)(self.labels_)  ## covert the problem into binary classifier
        clf = RandoForestClassifier()                                             ## run random fores and get important features for cluster
        clf.fit(X,binary_enc)


      sorted_feature_weight_idxes = np.arsort(list(np.arround(np.array(clf.feature_importances_),2)))[::-1]
      ordered_cluster_features = np.take_along_axis(np.array(self.ordered_feature_names),sorted_feature_weight_idxes, axis=0)
      ordered_cluster_feature_weights = np.take_along_axis(np.array(clf.feature_importances_), sorted_feature_weight_idxes, axis=0)

      cluster_feature_weights[label] = list(zip(ordered_cluster_features, list(np.around(np.array(ordered_cluster_feature_weights),2))))

      return cluster_feature_weights






In [ ]:
from plotly.express import px

def train_segmentation_model(train_data):

  kms=KMeansInterp(n_clusters=4,
                   ordered_feature_names = ["col1", "clo2"],
                    feature_importance_method = 'unsup2sup'
                  ).fit(train_data[["col1","col2"]])

  train_data["clusters"] = kms.labels_

  for i in range(len(train_data)):
    cluster_no = train_data.loc[i,"clusters"]
    train_data.at[i,"cluster_vector"] = kms.feature_importances_[cluster_no][:4]

  print(f"silhouette score:{silhhouette_score(df[["clo1","clo2"]], model.labels_)}")


def convert(tup,di):
  for a,b in tup:
    di.setdefault(a,[]).append(b)


def get_cluster_position(model, train_data):
  cluster_labels = pd.DataFrame()

  cluster_labels.at[0,"imp_features"] = model.feature_importances_[0][:4]
  cluster_labels.at[1,"imp_features"] = model.feature_importances_[1][:4]
  cluster_labels.at[2,"imp_features"] = model.feature_importances_[2][:4]
  cluster_labels.at[3,"imp_features"] = model.feature_importances_[3][:4]

  cluster_labels["cluster"] = cluster_labels.index
  cluster_labels["records"] = train_data.groupby(["clusters"]).size()

  tup_list = []
  for i in range(0,len(cluster_labels)):
    tup_list = tup_list + cluster_labels.loc[i,"imp_features"]

  dictionary = {}
  di = convert(tup_list, dictionary)

  cluster_df = pd.DataFrame(di)
  cluster_df = pd.concat([cluster_df,cluster_labels], axis=1)

  ## draw cluster ##
  fig = px.scatter_3d(cluster_df, x="clust1", y="clust2", z="clust3",
                      color="cluster", opacity=0.8, size='%guests', size_max=80)

  fig.show()

  return cluster_df